In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA

# 2022

In [2]:
team_stats = pd.read_csv('../../footy_data/league_one_teams_2022.csv')
games = pd.read_csv('../../footy_data/league_one_games_2022.csv')

In [3]:
games = games.loc[games['status'] == 'complete']
games
#Only include completed games

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Bolton Wanderers,Milton Keynes Dons,Martin Coy,1,0.0,0.0,...,2.45,3.35,2.75,1.29,1.80,2.80,4.95,1.69,2.15,University of Bolton Stadium (Bolton)
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Cambridge United,Oxford United,David Rock,1,0.0,0.0,...,3.55,3.40,2.05,1.35,1.95,3.10,5.50,1.80,2.00,Abbey Stadium (Cambridge- Cambridgeshire)
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Crewe Alexandra,Cheltenham Town,Ben Toner,1,0.0,0.0,...,2.25,3.05,3.40,1.39,2.10,3.40,6.75,1.87,1.91,Mornflake Stadium (Crewe- Cheshire)
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Doncaster Rovers,AFC Wimbledon,Samuel Barrott,1,0.0,0.0,...,2.40,3.25,2.95,1.34,1.95,3.10,5.75,1.77,2.00,The Keepmoat Stadium (Doncaster- South Yorkshire)
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Fleetwood Town,Portsmouth,Ben Speedie,1,0.0,0.0,...,2.95,3.15,2.40,1.48,2.25,3.90,7.50,2.05,1.74,Highbury Stadium (Fleetwood- Lancashire)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,1632854700,Sep 28 2021 - 6:45pm,complete,NaN,Plymouth Argyle,Crewe Alexandra,Neil Hair,10,2.2,0.4,...,1.86,3.50,3.60,1.33,2.01,3.20,6.00,1.78,1.95,Home Park (Plymouth)
113,1632854700,Sep 28 2021 - 6:45pm,complete,NaN,Rotherham United,AFC Wimbledon,Andy Haines,10,1.4,1.4,...,1.79,3.50,3.90,1.30,1.92,3.40,6.50,1.80,1.95,AESSEAL New York Stadium (Rotherham- South Yor...
114,1632854700,Sep 28 2021 - 6:45pm,complete,NaN,Shrewsbury Town,Wycombe Wanderers,Ross Joyce,10,1.4,1.0,...,2.90,3.30,2.23,1.33,2.03,3.75,7.50,1.91,1.91,Montgomery Waters Meadow (Shrewsbury- Shropshire)
115,1632854700,Sep 28 2021 - 6:45pm,complete,NaN,Sunderland,Cheltenham Town,Chris Sarginson,10,3.0,1.0,...,1.57,4.00,4.80,1.25,1.87,3.30,6.00,1.80,1.95,Stadium of Light (Sunderland)


Seeing how many games had goals and xg of over 2.5

In [4]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['xg_over/under'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 2.99, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['over/under_1.5'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 1.5, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [5]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100
percent_goals_1h

46.103896103896105

In [6]:
games['over/under'].value_counts()

0    61
1    55
Name: over/under, dtype: int64

In [7]:
games['xg_over/under'].value_counts()

0    70
1    46
Name: xg_over/under, dtype: int64

In [8]:
xg_view = games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]

In [11]:
xg_view[80:120]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
81,Cheltenham Town,Oxford United,1.11,1.68,1,0
82,Crewe Alexandra,Burton Albion,1.52,2.18,2,0
83,Doncaster Rovers,Morecambe,1.18,1.53,1,0
84,Fleetwood Town,Sunderland,1.58,1.85,2,2
85,Gillingham,Milton Keynes Dons,1.02,2.54,1,4
86,Lincoln City,Ipswich Town,0.97,0.85,0,1
87,Portsmouth,Cambridge United,1.39,1.20,1,2
88,Sheffield Wednesday,Shrewsbury Town,1.22,1.56,1,1
89,Wycombe Wanderers,Charlton Athletic,2.10,1.38,2,1
90,Crewe Alexandra,Morecambe,1.60,1.43,1,3


In [14]:
first_matchweek = games[0:12]

In [15]:
first_matchweek['over/under'].value_counts()

0    7
1    5
Name: over/under, dtype: int64

In [16]:
first_matchweek['over/under_1.5'].value_counts()

1    9
0    3
Name: over/under_1.5, dtype: int64

In [17]:
games['over/under_1.5'].value_counts()

1    87
0    29
Name: over/under_1.5, dtype: int64

In [18]:
games['over/under'].value_counts()

0    61
1    55
Name: over/under, dtype: int64

In [19]:
games['total_goals_at_half_time'].value_counts()

1    41
2    35
0    31
3     6
4     2
5     1
Name: total_goals_at_half_time, dtype: int64

In [20]:
games['home_team_goal_count_half_time'].sum()

75

In [21]:
games['away_team_goal_count_half_time'].sum()

67

# Teams

In [22]:
team_stats[['team_name', 'xg_for_avg_overall', 'xg_against_avg_overall']]

,team_name,xg_for_avg_overall,xg_against_avg_overall
0,Sunderland AFC,1.64,1.29
1,Burton Albion FC,1.60,1.39
2,Rotherham United FC,1.79,0.83
3,Sheffield Wednesday FC,0.81,1.11
4,Ipswich Town FC,1.16,1.00
5,Wigan Athletic FC,1.21,1.04
6,Charlton Athletic FC,1.19,1.31
7,Bolton Wanderers FC,1.60,1.20
8,Milton Keynes Dons FC,1.83,1.03
9,Doncaster Rovers FC,0.62,1.46


sos = total average of xg for xg against of opponents they have played

# Adding More Columns

In [23]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

In [24]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats

,common_name,xg_against_avg_overall
0,Sunderland,1.29
1,Burton Albion,1.39
2,Rotherham United,0.83
3,Sheffield Wednesday,1.11
4,Ipswich Town,1.00
5,Wigan Athletic,1.04
6,Charlton Athletic,1.31
7,Bolton Wanderers,1.20
8,Milton Keynes Dons,1.03
9,Doncaster Rovers,1.46


In [25]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [26]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [27]:
fill_stats(team_stats['common_name'][0])
fill_stats(team_stats['common_name'][1])
fill_stats(team_stats['common_name'][2]) 
fill_stats(team_stats['common_name'][3]) 
fill_stats(team_stats['common_name'][4]) 
fill_stats(team_stats['common_name'][5]) 
fill_stats(team_stats['common_name'][6]) 
fill_stats(team_stats['common_name'][7]) 
fill_stats(team_stats['common_name'][8]) 
fill_stats(team_stats['common_name'][9]) 
fill_stats(team_stats['common_name'][10]) 
fill_stats(team_stats['common_name'][11]) 
fill_stats(team_stats['common_name'][12]) 
fill_stats(team_stats['common_name'][13]) 
fill_stats(team_stats['common_name'][14]) 
fill_stats(team_stats['common_name'][15]) 
fill_stats(team_stats['common_name'][16]) 
fill_stats(team_stats['common_name'][17]) 
fill_stats(team_stats['common_name'][18]) 
fill_stats(team_stats['common_name'][19]) 
fill_stats(team_stats['common_name'][20]) 
fill_stats(team_stats['common_name'][21]) 
fill_stats(team_stats['common_name'][22]) 
fill_stats(team_stats['common_name'][23]) 